# **Word2Vec 분석하기**
1. **Frequency 분석** ( **문서의 개요** 살펴보기)
1. **Tf IDF** 분석 ( **특정 모집단 내 일부 문서의 특징** 찾기)
1. **Word2Vec 모델의** 분석 ( **적은 데이터로 키워드** 특징찾기)

## **1 PreProcessing 전처리 작업**
**Word2vec** 을 만들기 위해서는 **문장 데이터로** 전처리가 필요하다
1. **그냥 만든경우** 와, **문장을 선별한 경우를** 비교하여 성능 향상 시키기
1. **핵심 Token 을 찾고**, 이를 바탕으로 word2vec 결과값 필터링 하기 **(Core 찾기가 우선)**

In [1]:
import re
def dot_fixer(sent):
    dot_to_nun = re.compile(r"(?P<num>(\d+)).(?P<dot>(\d+))")
    nun_to_dot = re.compile(r"(?P<num>(\d+))=(?P<dot>(\d+))")
    post_temp  = dot_to_nun.sub("\g<num>=\g<dot>", sent)
    post_temp  = "\n".join(post_temp.split(".")).replace("^"," ").replace("["," ").replace("]"," ").replace("("," ").replace(")"," ")
    post_temp  = nun_to_dot.sub("\g<num>.\g<dot>", post_temp)
    return post_temp

In [2]:
# 크롤링 잘못된 소숫점 숫자를 치환후 줄바꿈 처리 모두 바꾸기
import pandas as pd
df = pd.read_csv("data/naverPost_muyong.csv")

from tqdm import tqdm
posts    = df.Posts.tolist()
df.Posts = [dot_fixer(_) for _ in tqdm(posts)]
df.to_csv("data/naverPost_muyong_fixed.csv")
print("dot fixed")

100%|██████████| 2333/2333 [00:51<00:00, 45.28it/s]


dot fixed


## **2 PreProcessing 2 유효문장 선별하기**
**Word2vec** 을 만들기 위해서는 **문장 데이터로** 전처리가 필요하다
1. **그냥 만든경우** 와, **문장을 선별한 경우를** 비교하여 성능 향상 시키기
1. **핵심 Token 을 찾고**, 이를 바탕으로 word2vec 결과값 필터링 하기 **(Core 찾기가 우선)**
1. 여기선 **문장내 단어가 3개 이상 출연한** 문장만 선별하기

In [10]:
import pandas as pd
df    = pd.read_csv("data/naverPost_muyong_fixed.csv")
key_posts = df.Posts.tolist()

result = []
for posts in tqdm(key_posts[:3]):
    posts_temp = []
    for post in posts.split("|"): # 500 개 문장
        for _ in post.split("\n"):
            if len(_.split(" ")) >= 3:
                posts_temp.append(_.strip())
    result.append("\n".join(posts_temp))

print(result[0][:300])

100%|██████████| 3/3 [00:00<00:00, 148.43it/s]

반찬 볶음 조림
부드러운 닭가슴살 장조림 만들기
본문 기타 기능
조림 밑반찬으로 대표적인 장조림
지방이 적고 단백질은 풍부한
을 이용해 만들어봤어요
고기를 잘게 찢어 넣어 식감은 더 부드럽게
과 제철을 맞이한
까지 넉넉하게 넣어 만들었어요
꽈리고추는 일반 고추에 비해 쭈글쭈글하니 못생겼지만
비타민A와 C 등의 영양가가 풍부하고 매운맛이 덜해 조림이나 찜으로 먹기에 좋아요
닭가슴살 장조림 재료
조리시간 1시간 이내 난이도 중
생 닭가슴살 2.0g 물 1L 5컵 청주 2큰술 마늘 4쪽 생강 마늘 크기 1톨
새송이버섯 2개 꽈리고추 2.


In [5]:
df.head(2)

,Unnamed: 0,Query,Posts
0,0,가슴살_계란_닭_장조림,반찬 볶음 조림\n부드러운 닭가슴살 장조림 만들기\n베리츄\n\n2017.9\n1....
1,1,가슴살_굴소스_닭_볶음밥,아임닭 7기 종료\n수비드 닭가슴살 요리 돈까스 볶음밥 만들기\n주니호맘\n\n20...


In [11]:
# 명사 Token 작업된 자료를 ssResport.txt 로 저장 
script_file = './w2v_script.txt'
with open(script_file, 'w', encoding='utf-8') as file:
    file.write(result[0])
    
print("전처리 블로그 내용의 저장 : {}".format(script_file))

전처리 블로그 내용의 저장 : ./w2v_script.txt


In [33]:
w2v_temp    = result[0].splitlines()
result_temp = []
for _ in w2v_temp:
    result_temp += _.split(" ")

from collections import Counter
for _ in df.Query[0].split("_"):
    print(_, Counter(result_temp)[_])

가슴살 16
계란 104
닭 31
장조림 342


## **1 Sentenced Text**
**Word2vec** 을 만들기 위해서는 **문장 데이터로** 전처리가 필요하다
1. **그냥 만든경우** 와, **문장을 선별한 경우를** 비교하여 성능 향상 시키기
1. **핵심 Token 을 찾고**, 이를 바탕으로 word2vec 결과값 필터링 하기 **(Core 찾기가 우선)**

In [36]:
%%time
from gensim.models import word2vec
data  = word2vec.LineSentence(script_file)
model = word2vec.Word2Vec(data, size=30, window=2, min_count=16, hs=1,
                          workers=4, iter=100, sg=1)
model.save(model_file)

Word 2 Vec 모델 파일 ./w2v_recipe.model 저장완료
CPU times: user 4.63 s, sys: 103 ms, total: 4.73 s
Wall time: 3.29 s


In [38]:
model.wv.most_similar(["가슴살", "계란", "닭", "장조림"], topn=20)

[('닭가슴살', 0.725135326385498),
 ('이용한', 0.6401489973068237),
 ('메추리알', 0.6393377780914307),
 ('요리', 0.6060360670089722),
 ('오늘', 0.5656761527061462),
 ('법', 0.5500603318214417),
 ('오늘은', 0.5277817845344543),
 ('닭가슴살과', 0.4969708025455475),
 ('장조림을', 0.4929303228855133),
 ('글입니다', 0.4797065854072571),
 ('만드는', 0.4728950560092926),
 ('만들기', 0.46609869599342346),
 ('위해', 0.4544617533683777),
 ('먹는', 0.44983047246932983),
 ('좋아요', 0.4398360848426819),
 ('꽈리고추', 0.43841955065727234),
 ('다이어트', 0.4327726662158966),
 ('닭고기', 0.4298769235610962),
 ('마늘', 0.42846792936325073),
 ('닭가슴살을', 0.42231449484825134)]

In [41]:
model.wv.most_similar(["가슴살", "계란", "닭", "장조림"], 
                      negative=['닭가슴살', '닭가슴살과', '닭가슴살을'],
                      topn=30)

[('요리', 0.6024991273880005),
 ('사용', 0.3841325342655182),
 ('법', 0.3550140857696533),
 ('글입니다', 0.35304853320121765),
 ('꽈리고추', 0.35070517659187317),
 ('이제', 0.3504452109336853),
 ('간단하게', 0.3490484952926636),
 ('위해', 0.3488353192806244),
 ('먹기', 0.3451535701751709),
 ('오늘', 0.34397000074386597),
 ('다이어트', 0.3195529878139496),
 ('메추리알', 0.31017300486564636),
 ('마늘', 0.3066818416118622),
 ('반찬으로', 0.2982470989227295),
 ('맛있어요', 0.2916230857372284),
 ('레시피', 0.28354838490486145),
 ('오늘은', 0.271423876285553),
 ('내용', 0.26687198877334595),
 ('장조림을', 0.26355087757110596),
 ('좋아요', 0.25986653566360474),
 ('글에', 0.2506791651248932),
 ('넣어서', 0.23929870128631592),
 ('만들', 0.23346063494682312),
 ('이용한', 0.23234985768795013),
 ('안', 0.2303457111120224),
 ('저작자', 0.22936281561851501),
 ('좋아하는', 0.22905974090099335),
 ('물에', 0.22215133905410767),
 ('자주', 0.21964050829410553),
 ('하나', 0.20812654495239258)]

In [34]:
import pandas as pd
posts = pd.read_csv("data/naverPost_muyong.csv")
posts.head(2)

,Query,Posts
0,가슴살_계란_닭_장조림,반찬 볶음 조림.부드러운 닭가슴살 장조림 만들기.베리츄..2017.9.1.22 32...
1,가슴살_굴소스_닭_볶음밥,아임닭 7기 종료.수비드 닭가슴살 요리 돈까스 볶음밥 만들기.주니호맘..2019.4...


In [4]:
post_temp = posts.loc[0]
post_temp_query = post_temp.Query.split("_")
post_temp_list = post_temp.Posts.split('|')

In [5]:
post_temp_query, post_temp_list[0][:300]

(['가슴살', '계란', '닭', '장조림'],
 '반찬 볶음 조림.부드러운 닭가슴살 장조림 만들기.베리츄..2017.9.1.22 32.URL 복사.이웃추가.본문 기타 기능.번역보기.조림 밑반찬으로 대표적인 장조림.지방이 적고 단백질은 풍부한.닭가슴살.을 이용해 만들어봤어요.고기를 잘게 찢어 넣어 식감은 더 부드럽게.거기에.새송이버섯.과 제철을 맞이한.꽈리고추.까지 넉넉하게 넣어 만들었어요.꽈리고추는 일반 고추에 비해 쭈글쭈글하니 못생겼지만.비타민A와 C 등의 영양가가 풍부하고 매운맛이 덜해 조림이나 찜으로 먹기에 좋아요^^.닭가슴살장조림.닭가슴살 장조림 재료.조리시간 1시간 이내 ')

In [6]:
# post 내용의 필터링
post_temp_list_blog = post_temp_list[0]
post_temp_list_blog = post_temp_list_blog.split('.')

In [7]:
import re

In [11]:
# 우선은 간단하게 기능을 추가한뒤
# 식재료 Stemming 보완이 필요한 듯 ex) 후춧가루, 후추, 양념장 과 같은...
#
".".join([sent for sent in post_temp_list_blog   if len(re.findall(r"\w+", sent)) >= 3])

'반찬 볶음 조림.부드러운 닭가슴살 장조림 만들기.본문 기타 기능.조림 밑반찬으로 대표적인 장조림.지방이 적고 단백질은 풍부한.을 이용해 만들어봤어요.고기를 잘게 찢어 넣어 식감은 더 부드럽게.과 제철을 맞이한.까지 넉넉하게 넣어 만들었어요.꽈리고추는 일반 고추에 비해 쭈글쭈글하니 못생겼지만.비타민A와 C 등의 영양가가 풍부하고 매운맛이 덜해 조림이나 찜으로 먹기에 좋아요^^.닭가슴살 장조림 재료.조리시간 1시간 이내 난이도 중.생 닭가슴살 200g 물 1L 5컵 청주 2큰술 마늘 4쪽 생강 마늘 크기 1톨.새송이버섯 2개 꽈리고추 200g 약 3줌.양념 고기 육수 300ml 1.5컵 양조간장 100ml 0.5컵 꿀 2큰술 표고버섯가루 1 2큰술 후춧가루 약간.괄호 속 기재된 컵 은 종이컵 기준입니다.꿀 대신 올리고당으로 대체 가능합니다.고기 삶은 물은 양념에 사용할 거예요.끓는 물에 닭 가슴살 물 1L 5컵 마늘 4쪽 생강 1톨 청주 2큰술을 넣고 삶아주세요.센 불에서 끓어오르면 중불에서 뚜껑을 덮어 20 25분 정도 삶으면 됩니다.삶은 물은 버리지 마세요.삶는 시간이 생각보다 오래 걸리기 때문에 이때 재료 손질 및 양념을 만들어두는 것이 좋아요.닭이 충분히 익었는지 확인할 땐 젓가락을 이용해 찔러보세요.젓가락이 푹 들어가고 핏물이 빠져나오지 않으면 충분하게 익은 거예요.고기 삶는 동안 꽈리고추를 흐르는 물에 깨끗하게 씻은 뒤 꼭지를 제거해주세요.포크를 이용해 구멍을 내주세요.양념이 잘 배도록 구멍을 내주는 것인데 작은 건 한번 긴 건 두 번 정도 콕 찔러주면 됩니다.길이가 너무 긴 것은 반으로 썰어주세요.꽈리고추 고르는 법.전체적으로 윤기가 나며 꼭지가 단단한 것을 고르면 되는데요.이때 몸통이 단단한 것은 좀 더 맵고 말랑한 것은 매운맛이 덜하니 기호에 따라 고르셔요^^.새송이버섯은 가로로 2등분 한 뒤 0.3cm 두께로 슬라이스해주세요.쫄깃한 식감의 새송이버섯은 꼭 넣어주세요.계란보다 더 맛 좋은 별미랍니다.슬라이스하는 것이 번거롭다면 미니새송이버섯을 사

## **1 Fast Text 모델링**
https://pypi.org/project/fasttext/
1. Gensim 보다 빠르다더라...
1. https://github.com/facebookresearch/fastText
1. **[Wiki 백과로 미리 훈련한 사전을 제공](https://dl.fbaipublicfiles.com/fasttext/vectors-aligned/wiki.ko.align.vec)**
1. [실습 예제자료를 다운 루는 블로그](https://inspiringpeople.github.io/data%20analysis/word_embedding/)

In [2]:
text_data = """반찬 볶음 조림.부드러운 닭가슴살 장조림 만들기.본문 기타 기능.조림 밑반찬으로 대표적인 장조림.
지방이 적고 단백질은 풍부한.을 이용해 만들어봤어요.고기를 잘게 찢어 넣어 식감은 더 부드럽게.과 제철을 맞이한.
까지 넉넉하게 넣어 만들었어요.꽈리고추는 일반 고추에 비해 쭈글쭈글하니 못생겼지만.
비타민A와 C 등의 영양가가 풍부하고 매운맛이 덜해 조림이나 찜으로 먹기에 좋아요^^.닭가슴살 장조림 재료.
조리시간 1시간 이내 난이도 중.생 닭가슴살 200g 물 1L 5컵 청주 2큰술 마늘 4쪽 생강 마늘 크기 1톨.
새송이버섯 2개 꽈리고추 200g 약 3줌.양념 고기 육수 300ml 1.5컵 양조간장 100ml 0.5컵 꿀 2큰술 표고버섯가루 1 2큰술
후춧가루 약간.괄호 속 기재된 컵 은 종이컵 기준입니다.꿀 대신 올리고당으로 대체 가능합니다.고기 삶은 물은 양념에 사용할 거예요."""

In [4]:
import fasttext
script_file = './w2v_script.txt'
# model = fasttext.skipgram(script_file, 'model', lr=0.1, dim=300)
# model = fasttext.skipgram('train.txt', 'model', lr=0.1, dim=300)
# model = fasttext.skipgram(text_data, 'model', lr=0.1, dim=300)
model = fasttext.train_unsupervised(script_file, model='skipgram', dim=300, lr=0.1)

In [ ]:
import fasttext

# Skipgram model :
model = fasttext.train_unsupervised('data.txt', model='skipgram')

# or, cbow model :
model = fasttext.train_unsupervised('data.txt', model='cbow')

In [ ]:
print(model.words)   # list of words in dictionary
print(model['king']) # get the vector of the word 'king'

In [1]:
# 학습결과 Model 로 저장하기
model.save_model("model_filename.bin")

In [ ]:
# 저장된 Model 객체 불러오기
model = fasttext.load_model("model_filename.bin")

In [ ]:
import fasttext
model = fasttext.train_supervised('data.train.txt')
print(model.words)
print(model.labels)

In [ ]:
model.words         # equivalent to model.get_words()
model.labels        # equivalent to model.get_labels()

In [ ]:
model['king']       # equivalent to model.get_word_vector('king')
'king' in model     # equivalent to `'king' in model.get_words()`

In [ ]:
# 개별 Token 의 가중치 확인하기
def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

print_results(*model.test('test.txt'))

In [ ]:
# 학습 모델을 활용하여 판단하기
model.predict("Which baking dish is best to bake a banana bread ?")

In [ ]:
# 출력할 결과값을 사용자가 지정 가능 (k=3)
model.predict("Which baking dish is best to bake a banana bread ?", k=3)

In [ ]:
# If you want to predict more than one sentence you can pass an array of strings 
model.predict(["Which baking dish is best to bake a banana bread ?", "Why not put knives in the dishwasher?"], k=3)

In [ ]:
# with the previously trained `model` object, call :
model.quantize(input='data.train.txt', retrain=True)

# then display results and save the new model :
print_results(*model.test(valid_data))
model.save_model("model_filename.ftz")

## **1 Naver Link 목록**
OpenAPI 로 수집한 결과값 저장된 목록 List 원본의 수정보완
1. Naver OPEN API 결과 **Token 이 유효값인지** 확인 가능했다 (Oh!!)
1. 네이버 블로그 목록만 정리한 DataBase 만들기
1. 목록을 4개로 나워서 2개의 서버에서 돌리며 저장하기

In [3]:
import pandas as pd
df = pd.read_csv('data/naver_posts_sample.csv')
df.head(3)

,Query,Posts
0,가슴살_계란_닭_장조림,반찬 볶음 조림.부드러운 닭가슴살 장조림 만들기.베리츄..2017.9.1.22 32...
1,간장_불고기_제육_파채,오늘의 식탁.[반조리 밥아저씨] 12월 12일 식탁 버섯제육간장불고기 파채 감자고추...
2,감자_국_수제비,나의이야기.아침에 먹고 남은 감자국으로 수제비 만드는법 하기 위해 수제비 반죽만들기...


In [14]:
querys = sorted(list(set("_".join(df.Query.tolist()).split("_"))))
print(len(querys)), " ".join(querys)

86


(None,
 '가슴살 간장 감자 강정 겉절이 계란 계란말이 고추 고추장 골뱅이 국 굴 김치 깨 꼬치 꽃게 너비아니 달래 닭 데리야끼 도라지생채 돈 돈육 된장국 두부 들깨 떡국 만두 매콤 무 무침 물 미나리 미소 미역 버섯 볶음 부추 불고기 브로콜리 사골 사태 상추 새송이 새우젓 샐러드 소고기 소스 수제비 순두부 시금치나물 야채 양상추 양파 어묵 엿 오이 오징어 우동 유자 장국 장조림 장포 적 제육 조림 짬뽕 찜 찹쌀 채 채소 청경채 초무침 치어 치즈 칠리 콩나물 탕 탕수 파채 팽이 해물탕 햄 황도 후랑크 후르츠')

## **1 Naver Link 목록**
OpenAPI 로 수집한 결과값 저장된 목록 List 원본의 수정보완
1. Naver OPEN API 결과 **Token 이 유효값인지** 확인 가능했다 (Oh!!)
1. 네이버 블로그 목록만 정리한 DataBase 만들기
1. 목록을 4개로 나워서 2개의 서버에서 돌리며 저장하기